# Example of Search Space creation using composite hyperparameters
___

#### Before diving into example, let me clarify some key notions used here:
- Search Space: combination of Hyperparameters and their relationships. Even one Hyperparameter could define a Search Space by itself.
- Hyperparameter: One of possible characteristics of Search Space, could be one of following types: Categorical (Nominal, Ordinal) and Numeric (Integer, Float).


#### Imagine, you need to create a Search Space for metaheuristics search.

On the **top level**, you define a **meta-heuristics** by their **names**.
On the **level below** you define child parameters for each meta-heuristics. These parameters could require definition of different parameters, if some specific values are taken.

For instance, for python-based meta-heuristic of type "Evolution Strategy" (**ES**), one have to define following parameters (that later become Hyperparameters of Search Space):
- 'mu' - Integer, defines number of parents which create offspring;
- 'lambda' - Integer, number of child Solutions created by *mu* parents;
- 'elitist' - the selection algorithm type, defines (lambda+mu) or (lambda, mu) ES type;
- 'mutation type' - either "Permutation" or "Scramble" mutation - strategy to perform mutation;
- 'mutation probability' floating number between 0 and 1.

For such parameters as "mutation type" and "crossover type" one also should specify probability of applying opperation.
This defines a shape of Search Space as a tree.
##### See graphical illustration of Hyperparameters, their types and dependencies:

In [1]:
from IPython.display import IFrame
IFrame("https://miro.com/app/embed/o9J_kvvl4xw=/?", width=900, height=650)

## Search Space: Step-wise construction
#### Some key points, that should be kept in mind:
- Currently, only the Categorical Hyperparameter could have a children.
- All hyperparameters, that could appear simmultaneously in Search Space must have different names (to avoid collitions).

First, lets create a root Hyperparameter.
We are choosing one of 3 different types of meta-heuristics: python-based Evolutionary Strategy and Simmulated Annealing and java-based Evolution strategy.

Based chosen meta-heuristic, different parameters could be exposed, so root of Search Space is deffinitely type of meta-heuristic. The best way to reflect this information is to define it as a Categorical Hyperparameter with tree different categories - "jMetalPy.SimulatedAnnealing", "jMetalPy.EvolutionStrategy" and "jMetal.EvolutionStrategy".
Categorical data type by itself could be Ordinal or Nominal. Some predictional models could utilize this information in their prediction.
Characteristics of each type:
- Ordinal data type defines notions of 'closeness' and 'order' between categories. It is possible to derive 'mean' and 'mode' of samples bunch. Examples could be ["cold", "normal", "hot"] or ["bad", "neutral", "good"].
- Nominal data type does not implies neither of 'closeness' not 'order' notions. Instead of 'mean' one could only derive 'mode' from bunch of information.

It is clear, that type of meta-heuristic is a nominal data and those Nominal Hyperparameter type should be used:

In [1]:
%cd ..
from core_entities.search_space import NominalHyperparameter, IntegerHyperparameter, FloatHyperparameter

/media/sem/B54BE5B22C0D3FA81/TUD/Master/code/experiments


In [2]:
mh_type = NominalHyperparameter(
    name="low level heuristic",                            
    categories=["jMetalPy.SimulatedAnnealing", "jMetalPy.EvolutionStrategy", "jMetal.EvolutionStrategy"]
)
print(mh_type)

NominalHyperparameter 'low level heuristic'.
├ Default category: 'jMetalPy.EvolutionStrategy'.
├ Categories:
├  jMetalPy.SimulatedAnnealing:
├  jMetalPy.EvolutionStrategy:
├  jMetal.EvolutionStrategy:


### Create Hyperparameters
Now lets define child hyperparameters for python-based evolution strategy meta-heuristic.

The parameters for all other meta-heuristics are defined in the same way.

In [6]:
py_es_mu = IntegerHyperparameter("mu", 1, 1000, 500)
py_es_lambda = IntegerHyperparameter("lambda_", 1, 1000, 500)
py_es_elitist = NominalHyperparameter('elitist', ['True', 'False'], 'False')
py_es_mut = NominalHyperparameter("mutation_type", ['PermutationSwapMutation', 'ScrambleMutation'], 'PermutationSwapMutation')
py_es_mut_prob = FloatHyperparameter("mutation_probability", 0, 1, 0.5)

### Add parent-child relationships

Since *mutation type* hyperparameter defines child *mutation probability* it also should be defined and added to *mutation type* as a child. In the same way the parameters are linked to their algorith types. 
**Note**, one could skip 'activation_categories' parameter when adding child, if child should appear behind each category.

In [8]:
py_es_mut.add_child_hyperparameter(py_es_mut_prob)
for hp in (py_es_mu, py_es_lambda, py_es_elitist, py_es_mut):
    mh_type.add_child_hyperparameter(hp, activation_category='jMetalPy.EvolutionStrategy')

Now lets check a structure of the constructed so far search space.

In [10]:
print(mh_type)

NominalHyperparameter 'low level heuristic'.
├ Default category: 'jMetalPy.EvolutionStrategy'.
├ Categories:
├  jMetalPy.SimulatedAnnealing:
├  jMetalPy.EvolutionStrategy:
├+   IntegerHyperparameter 'mu'
├    | Lower boundary: 1, upper boundary: 1000.
├    | Default value: 500.
├+   IntegerHyperparameter 'lambda_'
├    | Lower boundary: 1, upper boundary: 1000.
├    | Default value: 500.
├+   NominalHyperparameter 'elitist'.
├    ├ Default category: 'False'.
├    ├ Categories:
├    ├  True:
├    ├  False:
├+   NominalHyperparameter 'mutation_type'.
├    ├ Default category: 'PermutationSwapMutation'.
├    ├ Categories:
├    ├  PermutationSwapMutation:
├    ├+   FloatHyperparameter 'mutation_probability'
├    ├    | Lower boundary: 0.0, upper boundary: 1.0.
├    ├    | Default value: 0.5.
├    ├  ScrambleMutation:
├    ├+   FloatHyperparameter 'mutation_probability'
├    ├    | Lower boundary: 0.0, upper boundary: 1.0.
├    ├    | Default value: 0.5.
├  jMetal.EvolutionStrategy:


As you may see, the jMetalPy.Evolution strategy hyperparameter now contains several child parameters with the displayed values.

Let's define the parameters for other meta-heuristic and link them together. 

In [11]:
# simulated annealing
py_sa_mut = NominalHyperparameter("mutation_type", ['PermutationSwapMutation', 'ScrambleMutation'], 'PermutationSwapMutation')
py_sa_mut_prob = FloatHyperparameter("mutation_probability", 0, 1, 0.5)
py_sa_mut.add_child_hyperparameter(py_sa_mut_prob)
mh_type.add_child_hyperparameter(py_sa_mut, activation_category='jMetalPy.SimulatedAnnealing')

# evolution strategy
j_es_mu = IntegerHyperparameter(name="mu", lower=1, upper=1000, default_value=500)
j_es_lambda = IntegerHyperparameter("lambda", 1, 1000, 500)
j_es_elitist = NominalHyperparameter("elitist", ['True', 'False'], 'False')
j_es_mutation_prob = FloatHyperparameter("mutation_probability", 0, 1, 0.5)
for hp in (j_es_mu, j_es_lambda, j_es_elitist, j_es_mutation_prob):
    mh_type.add_child_hyperparameter(hp, activation_category='jMetal.EvolutionStrategy')

In [13]:
# lets check the resulted search space
print(mh_type)

NominalHyperparameter 'low level heuristic'.
├ Default category: 'jMetalPy.EvolutionStrategy'.
├ Categories:
├  jMetalPy.SimulatedAnnealing:
├+   NominalHyperparameter 'mutation_type'.
├    ├ Default category: 'PermutationSwapMutation'.
├    ├ Categories:
├    ├  PermutationSwapMutation:
├    ├+   FloatHyperparameter 'mutation_probability'
├    ├    | Lower boundary: 0.0, upper boundary: 1.0.
├    ├    | Default value: 0.5.
├    ├  ScrambleMutation:
├    ├+   FloatHyperparameter 'mutation_probability'
├    ├    | Lower boundary: 0.0, upper boundary: 1.0.
├    ├    | Default value: 0.5.
├  jMetalPy.EvolutionStrategy:
├+   IntegerHyperparameter 'mu'
├    | Lower boundary: 1, upper boundary: 1000.
├    | Default value: 500.
├+   IntegerHyperparameter 'lambda_'
├    | Lower boundary: 1, upper boundary: 1000.
├    | Default value: 500.
├+   NominalHyperparameter 'elitist'.
├    ├ Default category: 'False'.
├    ├ Categories:
├    ├  True:
├    ├  False:
├+   NominalHyperparameter 'mutation_

### Search Space construction using json description
One could also provide json description of Search Space and use created tool to build Search Space from json description:

In [16]:
from core_entities.search_space import from_json
mh_type_json_loaded = from_json("./Resources/HyperHeuristic/HHData.json")

# It is also possible to compare the equality of search spaces, recursively comparing each underlying hyperparameter (name, values, children, etc.).
# Lets check if the created in step-wise approach and its description in JSON file are the same:
print(mh_type == mh_type_json_loaded)

True


# Configuration: creation and sampling
After defining the Search Space, lets check the Configurations sampling. It is done iteratively, level-per-level, each time traversing deeper in Search Space. For the Configuration representation we use the **mutable mapping**, each time addding the next layer of parameters.

For instance, in our use-case, on the first level the metaheurisitic type will be selected:

In [21]:
import numpy as np
np.random.seed(1) # reproductivity of the same results.
config = {}
mh_type.generate(config)
print(config)

{'low level heuristic': 'jMetalPy.EvolutionStrategy'}


After selecting the meta-heuristic type, its respective child hyperparameters will be exposed and sampled:

In [22]:
mh_type.generate(config)
print(config)

{'low level heuristic': 'jMetalPy.EvolutionStrategy', 'mu': 236, 'lambda_': 909, 'elitist': 'True', 'mutation_type': 'ScrambleMutation'}


The process of configuration sampling is defined in such a way that the sampling on each level guarantees to provide a valid parameter values. Also, after sampling on the previous level, those parameters will not be altered, therefore, in the search space of maximal depth **N**, after maximally N *generate* calls, the sampling will be finished.

For better usability, we added the *validate* method to control the prediction process. It may be called recursively, to estimate whether the configuration sampling is finished. Or it may be executed on a specific parameter instance without recursive calling to check if underlying paramers' value was not violated.

In [23]:
print(config)
while not mh_type.validate(config, recursive=True):
    mh_type.generate(config)
    print(config)

{'low level heuristic': 'jMetalPy.EvolutionStrategy', 'mu': 236, 'lambda_': 909, 'elitist': 'True', 'mutation_type': 'ScrambleMutation'}
{'low level heuristic': 'jMetalPy.EvolutionStrategy', 'mu': 236, 'lambda_': 909, 'elitist': 'True', 'mutation_type': 'ScrambleMutation', 'mutation_probability': 0.12812444792935673}


### Configuration: description

On each point, one could need a description of available information in Configuration to perform different kinds of actions, for instance in machine learning pipelines, such a Configuration should be encoded to numeric data (encoding), and numeric data could be altered for better fit into models (data transformation):

In [24]:
mh_type.describe(config)

OrderedDict([('low level heuristic',
              {'hyperparameter': NominalHyperparameter 'low level heuristic'.
               ├ Default category: 'jMetalPy.EvolutionStrategy'.
               ├ Categories:
               ├  jMetalPy.SimulatedAnnealing:
               ├+   NominalHyperparameter 'mutation_type'.
               ├    ├ Default category: 'PermutationSwapMutation'.
               ├    ├ Categories:
               ├    ├  PermutationSwapMutation:
               ├    ├+   FloatHyperparameter 'mutation_probability'
               ├    ├    | Lower boundary: 0.0, upper boundary: 1.0.
               ├    ├    | Default value: 0.5.
               ├    ├  ScrambleMutation:
               ├    ├+   FloatHyperparameter 'mutation_probability'
               ├    ├    | Lower boundary: 0.0, upper boundary: 1.0.
               ├    ├    | Default value: 0.5.
               ├  jMetalPy.EvolutionStrategy:
               ├+   IntegerHyperparameter 'mu'
               ├    | Lower bounda